In [1]:
from pathlib import Path
import random
import re

from Bio import SeqIO
import numpy as np

In [2]:
THRESHOLD = 0.2
FILTER = False
QUERY = "ORF"
rna_path = "../data/S288C_reference_genome_R62-1-1_20090218/orf_coding_all_R62-1-1_20090220.fasta" # mRNA
ref_path = "../data/S288C_reference_genome_R62-1-1_20090218/S288C_reference_sequence_R62-1-1_20090218.fsa"

In [3]:
def parse_gene_infos(records):
    """parse gene info from fasta file (for R62 genome)"""
    infos = []
    for rec in records:
        gene_id = rec.id
        name = rec.description.split(" ")[1]
        chr = "chr" + rec.description.split(" ")[4]
        coords = rec.description.split(" ")[6]
        starts = []
        ends = []
        for c in coords[:-1].split(","):
            start = int(c.split("-")[0]) - 1
            end = int(c.split("-")[1])
            if start > end:
                start, end = end, start
            starts.append(start)
            ends.append(end)
        infos.append([gene_id, name, chr, starts, ends])
    return infos

def filter(records, query):
    regex = re.compile(query)
    for rec in records:
        if bool(regex.search(rec.description)):
            yield rec

def find_save_rec(transcripts, gene_name, transcript_id):
    for rec in transcripts:
        if rec.id == transcript_id:
            Path(f"../output/{gene_name}").mkdir(parents=True, exist_ok=True)
            with open(f"../output/{gene_name}/nucleotide.fasta", "w") as f:
                f.write(f">{rec.description}\n{rec.seq}\n")
            with open(f"../output/{gene_name}/protein.fasta", "w") as f:
                f.write(f">{rec.description} translated\n{rec.seq.translate()}\n")
            return rec


def get_signals(chr, starts, ends):
    signals = []
    with open(f"../data/dms_signal/processed/{chr}.bin", "rb") as f:
        s = np.load(f)
    for start, end in zip(starts, ends):
        signals += list(s[start:end])
    return signals


def make_save_constraints(signals, rec, gene_name, threshold=0.2):
    constraints = [
        "x" if signals[i] > threshold
        else "." for i in range(len(signals))
    ]
    with open(f"../output/{gene_name}/constrained.fasta", "w") as f:
        f.write(f">{rec.description}\n{rec.seq}\n{''.join(constraints)}\n")

In [4]:
with open(rna_path) as f:
    records = list(SeqIO.parse(f, "fasta"))
if FILTER:
    records = filter(records, QUERY)
infos = parse_gene_infos(records)
infos[:5]

[['YAL001C', 'TFC3', 'chrI', [151099, 147596], [151167, 151007]],
 ['YAL002W', 'VPS8', 'chrI', [143708], [147533]],
 ['YAL003W', 'EFB1', 'chrI', [142175, 142621], [142255, 143162]],
 ['YAL004W', 'YAL004W', 'chrI', [140761], [141409]],
 ['YAL005C', 'SSA1', 'chrI', [139505], [141432]]]

In [5]:
genes_to_process = random.sample(infos, k=10)
[id for id, _, _, _, _ in genes_to_process]

['YBR121C-A',
 'YMR031W-A',
 'YOR068C',
 'YIL053W',
 'YFR048W',
 'YGR296C-A',
 'YPR179C',
 'YGR038C-A',
 'YMR115W',
 'YPR020W']

In [6]:
for id, name, chr, starts, ends in genes_to_process:
    transcripts = SeqIO.parse(rna_path, "fasta")
    if FILTER:
        transcripts = filter(transcripts, QUERY)
    rec = find_save_rec(transcripts, name, id)
    signals = get_signals(chr, starts, ends)
    make_save_constraints(signals, rec, name, THRESHOLD)